In [1]:
import torch
import torch.nn as nn
import numpy as np
import snntorch as snn
import matplotlib.pyplot as plt
from snntorch import spikegen
from snntorch import surrogate
from snntorch import utils
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tensorflow as tf
import random
import csv
from forest_fire_snn import SNN
import pandas as pd

Written 100000 rows to firetest_data.csv
Fire rows: 50279
No-fire rows: 49721


In [5]:
# Create an instance of SNN Object
net = SNN()

# Prepare CSV data
df = pd.read_csv('firetest_data.csv')
features = df[['Temp', 'Audio', 'Humidity', 'CO2']]
labels = df['Fire']

# Normalize features
df['Temp'] = df['Temp'] / 100
df['Humidity'] = df['Humidity'] / 100
df['CO2'] = df['CO2'] / 5000

X = df[['Temp', 'Audio', 'Humidity', 'CO2']]
Y = df['Fire'].to_numpy()

X_np = X.to_numpy()
Y_np = Y
X_tensor = torch.tensor(X_np, dtype=torch.float32)
X_tensor = X_tensor.clamp(0,1)
Y_tensor = torch.tensor(Y_np, dtype=torch.long)

# Convert features to spike trains
num_steps = 25
def to_spike_trains(X_tensor, num_steps=25):
    num_samples, num_features = X_tensor.shape
    spike_data = torch.zeros((num_samples, num_steps, num_features))
    
    for i in range(num_samples):
        for t in range(num_steps):
            spike_data[i, t] = torch.bernoulli(X_tensor[i])
    return spike_data

fire_train = to_spike_trains(X_tensor, num_steps=num_steps)

class FireDataset(torch.utils.data.Dataset):
    def __init__(self, spike_data, labels):
        self.spike_data = spike_data
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.spike_data[idx], self.labels[idx]
    
dataset = FireDataset(fire_train, Y_tensor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Training Loop
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)

num_epochs = 50

for epoch in range(num_epochs):
    net.train()
    total_loss, total_correct, total_n = 0.0, 0, 0
    
    for data, targets in train_loader:
        data = data.permute(1, 0, 2)
        utils.reset(net)
        spk_rec, mem_rec = net(data)
        logits = spk_rec.sum(dim=0)
        
        loss = loss_fn(logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        bs = targets.size(0)
        total_loss += loss.item() * bs
        total_correct += (logits.argmax(dim=1) == targets).sum().item()
        total_n += bs
    print(f"Epoch {epoch+1}/{num_epochs} | Loss={total_loss/total_n:.4f} | Acc={total_correct/total_n:.3f}")


Epoch 1/50 | Loss=0.2311 | Acc=0.903
Epoch 2/50 | Loss=0.2086 | Acc=0.915
Epoch 3/50 | Loss=0.2050 | Acc=0.916
Epoch 4/50 | Loss=0.2025 | Acc=0.917
Epoch 5/50 | Loss=0.1989 | Acc=0.918
Epoch 6/50 | Loss=0.1966 | Acc=0.919
Epoch 7/50 | Loss=0.1944 | Acc=0.919
Epoch 8/50 | Loss=0.1931 | Acc=0.920
Epoch 9/50 | Loss=0.1940 | Acc=0.919
Epoch 10/50 | Loss=0.1927 | Acc=0.920
Epoch 11/50 | Loss=0.1917 | Acc=0.920
Epoch 12/50 | Loss=0.1904 | Acc=0.921
Epoch 13/50 | Loss=0.1906 | Acc=0.920
Epoch 14/50 | Loss=0.1895 | Acc=0.921
Epoch 15/50 | Loss=0.1896 | Acc=0.921
Epoch 16/50 | Loss=0.1896 | Acc=0.921
Epoch 17/50 | Loss=0.1878 | Acc=0.921
Epoch 18/50 | Loss=0.1886 | Acc=0.922
Epoch 19/50 | Loss=0.1884 | Acc=0.921
Epoch 20/50 | Loss=0.1883 | Acc=0.921
Epoch 21/50 | Loss=0.1871 | Acc=0.921
Epoch 22/50 | Loss=0.1871 | Acc=0.921
Epoch 23/50 | Loss=0.1874 | Acc=0.921
Epoch 24/50 | Loss=0.1879 | Acc=0.921
Epoch 25/50 | Loss=0.1865 | Acc=0.922
Epoch 26/50 | Loss=0.1875 | Acc=0.922
Epoch 27/50 | Loss=0.

In [ ]:
from torch.utils.data import random_split, DataLoader
import torch
from snntorch import utils

def make_train_val_loaders(dataset, batch_size=64, val_frac=0.2, seed=42):
    n = len(dataset)
    val_n = int(val_frac * n)
    train_n = n - val_n

    train_ds, val_ds = random_split(
        dataset,
        [train_n, val_n],
        generator=torch.Generator().manual_seed(seed)
    )

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

def eval_acc_loss(net, loader, loss_fn, device=None):
    """
    net: your SNN model
    loader: train_loader or val_loader
    loss_fn: e.g. nn.CrossEntropyLoss()
    device: optional torch.device; if provided, moves tensors to device
    """
    net.eval()
    total_loss, total_correct, total_n = 0.0, 0, 0

    with torch.no_grad():
        for data, targets in loader:
            # data shape from dataset: (batch, time, features)
            # you want: (time, batch, features)
            data = data.permute(1, 0, 2)

            if device is not None:
                data = data.to(device)
                targets = targets.to(device)

            utils.reset(net)
            spk_rec, mem_rec = net(data)   # spk_rec: (time, batch, num_outputs)

            logits = spk_rec.sum(dim=0)    # (batch, num_outputs)
            loss = loss_fn(logits, targets)

            bs = targets.size(0)
            total_loss += loss.item() * bs
            total_correct += (logits.argmax(dim=1) == targets).sum().item()
            total_n += bs

    return total_loss / total_n, total_correct / total_n


train_loader, val_loader = make_train_val_loaders(dataset, batch_size=64, val_frac=0.2, seed=42)
train_loss, train_acc = eval_acc_loss(net, train_loader, loss_fn)
val_loss, val_acc     = eval_acc_loss(net, val_loader, loss_fn)
print(f"Train: loss={train_loss:.4f}, acc={train_acc:.3f} | Val: loss={val_loss:.4f}, acc={val_acc:.3f}")



TypeError: eval_acc_loss() missing 1 required positional argument: 'loader'